In [121]:
# Bibliothèques
from hubspot.crm.contacts import SimplePublicObjectInput
from hubspot.crm.contacts.exceptions import ApiException
from hubspot import HubSpot


In [122]:
# Connection à l'API
api_client = HubSpot(access_token=token)

# Récupération des infos des deals

In [134]:
# Récupération du montant total des deals sans pondération
def total_deal_value():
    
    # Recupération des ids des deals (si on considère que tous les deals sont associés à une seule entreprise)
    deals_id=[]
    deals=api_client.crm.deals.get_all()
    for deal in deals:
        deals_id.append(deal.to_dict()["id"])

    montant_deals=0
    for id in deals_id:
        deal_data=api_client.crm.deals.basic_api.get_by_id(id,properties=["amount"]).to_dict()
        montant_deals+=float(deal_data["properties"]["amount"])
    return montant_deals

In [124]:
# Calcul du montant total des deals avec pondération

def total_deal_value_weighted():

    # Recupération des ids des deals (si on considère que tous les deals sont associés à une seule entreprise)
    deals_id=[]
    deals=api_client.crm.deals.get_all()
    for deal in deals:
        deals_id.append(deal.to_dict()["id"])
    
    # Calcul du montant total des deals avec pondération
    montant_deals_pond=0
    for id in deals_id:
        deal_data=api_client.crm.deals.basic_api.get_by_id(id,properties=["amount","hs_deal_stage_probability"]).to_dict()
        montant_deals_pond+=float(deal_data["properties"]["amount"]) * float(deal_data["properties"]["hs_deal_stage_probability"])
    return montant_deals_pond


### Update du montant d'un deal


In [125]:
def update_deal(deal_id,amount):
    try:
        update_montant_deal = SimplePublicObjectInput(
                    properties={"amount":amount})
        api_client.crm.deals.basic_api.update(simple_public_object_input=update_montant_deal,deal_id=deal_id)
        return print(f"Deal {deal_id} updated")
    except ApiException as deal:
        return print("Exception when updatin deal: %s\n" %deal)  
    

# Contacts

In [126]:
# Calcul du nombre de contacts dont En poste = Oui

def count_en_poste(etat="Oui"):

    # Récupération des id des contacts
    contacts_id=[]
    contacts=api_client.crm.contacts.get_all()
    for contact in contacts:
        contacts_id.append(contact.to_dict()["id"])

    # Calcul du nombre de contact selon l'état de l'attribut en poste
    cpt_en_poste=0
    for id in contacts_id:
        contact_data=api_client.crm.contacts.basic_api.get_by_id(id,properties=["en_poste"]).to_dict()
        if contact_data["properties"]["en_poste"] == etat:
            cpt_en_poste+=1
    return cpt_en_poste

In [127]:
# Propriétaire actuel

def update_owner():
    owner_id=api_client.crm.companies.basic_api.get_by_id(7496989169,properties=["hubspot_owner_id"]).to_dict()
    owner_id=owner_id["properties"]["hubspot_owner_id"]
    owner_data=api_client.crm.owners.owners_api.get_by_id(owner_id).to_dict()
    return [owner_data["first_name"],owner_data["last_name"]]

# Mise à jour des infos de l'entreprise

In [128]:
# Mise à jour des infos de l'entreprise

def update_company(comp_id="7496989169"):
        try:
                update_montant_tot_deals = SimplePublicObjectInput(properties={
                "valeur_totale_des_deals":total_deal_value() ,
                "valeur_totale_des_deals_ponderee":total_deal_value_weighted(),
                "nombre_de_points_de_contacts_en_poste": count_en_poste(),
                "nom_du_dirigeant":update_owner()[1],
                "prenom_du_dirigeant":update_owner()[0]})
                api_client.crm.companies.basic_api.update(simple_public_object_input=update_montant_tot_deals,company_id=comp_id)
                return print("company updated")
        except ApiException as update:
                return print("Exception when updating company: %s\n" % update)
update_company()
        

company updated


In [130]:
# Récupération des infos de l'entreprise

def comp_info(comp_id="7496989169"):
    company_data=api_client.crm.companies.basic_api.get_by_id(comp_id,properties=["name","city","country","industry",
                                                                             "nom_du_dirigeant","prenom_du_dirigeant",
                                                                             "numberofemployees","valeur_totale_des_deals",
                                                                             "valeur_totale_des_deals_ponderee","num_associated_deals",
                                                                             "nombre_de_points_de_contacts_en_poste",
                                                                             "nombre_de_points_de_contacts_en_poste"]).to_dict()
    return company_data["properties"]